## Environment setup

In [1]:
!pip install shap lime ollama python-dotenv
#!pip install "numpy==1.26.4" --force-reinstall
#!pip install "shap<0.50" lime ollama python-dotenv



## Imports, path setup, pipeline reload

In [2]:
import os
import sys
import importlib

# If the notebook is inside the `demo/` folder,
# the project root is one level up.
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

# Classic ML stack
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Make sure we reload the latest version of the pipeline module
import src.pipeline
importlib.reload(src.pipeline)

from config.settings import FrameworkConfig
from src.pipeline import XAINLGPipeline
from src.nlg.few_shot_generator import FewShotGenerator
from src.nlg.ollama_client import ollama_llm_call


## Dataset, model, pipeline, basic SHAP explanation

In [3]:
# 1. Load Breast Cancer Wisconsin dataset
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = list(data.feature_names)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Train a simple RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 3. Create framework config and pipeline
config = FrameworkConfig(verbose=True)
config.explainer.top_k_features = 5  # how many top features to use for NLG

config.nlg.debug_print_prompt = True


pipeline = XAINLGPipeline(
    model=model,
    data=X_train,
    feature_names=feature_names,
    config=config,
)

print(f"Train size: {len(X_train)}, test size: {len(X_test)}, features: {len(feature_names)}")

# 4. Select one instance to explain
instance = X_test[0]
raw_pred = int(model.predict(instance.reshape(1, -1))[0])
print("Raw prediction (0=malignant, 1=benign):", raw_pred)

# 5. Run pipeline only up to XAI (no NLG yet)
result = pipeline.explain_instance(
    instance,
    method="shap",
    generate_text=False,   # we handle NLG manually in the next cells
)

print("Available keys in result:", result.keys())


INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30


Train size: 455, test size: 114, features: 30
Raw prediction (0=malignant, 1=benign): 1
Available keys in result: dict_keys(['instance', 'method', 'explanation', 'base_value', 'prediction', 'ranked_features', 'statements'])


## Few-Shot NLG with Ollama (expert vs layman)

In [4]:
# === Layer 3: NLG with Ollama (Few-Shot) ===

# We slightly lower the temperature for more stable outputs.
config.nlg.temperature = 0.2
nlg = FewShotGenerator(config.nlg, llm_call_fn=ollama_llm_call)

# Take the top-k features from the SHAP explanation
top_k = result["ranked_features"][: config.explainer.top_k_features]
print("Top-k ranked features:", top_k)

# Build "direction" labels from the contribution sign
directions = []
for _, v in top_k:
    if v > 0:
        directions.append("supports")
    elif v < 0:
        directions.append("contradicts")
    else:
        directions.append("neutral")

# Map raw prediction 0/1 to domain-specific labels (EN)
raw_pred = int(result["prediction"])
pred_expert_en = "malignant tumor" if raw_pred == 0 else "benign tumor"
pred_layman_en = "high risk of breast cancer" if raw_pred == 0 else "low risk of breast cancer"

# Shared context used by all generators
base_context = {
    "features": [f for f, _ in top_k],
    "values": [float(v) for _, v in top_k],
    "directions": directions,
    "method": result.get("method", "shap"),
}

# 1) Context for an expert-level explanation
context_expert = {
    **base_context,
    "prediction": pred_expert_en,
    "audience": "expert",   # FewShotGenerator will switch to technical style
}

# 2) Context for a layman-friendly explanation
context_layman = {
    **base_context,
    "prediction": pred_layman_en,
    "audience": "layman",   # FewShotGenerator will switch to simple language
}

# --- Print one example prompt for Few-Shot (expert) ---
# few_shot_prompt_expert = nlg.build_few_shot_prompt(context_expert, style="expert")
# print("=== FEW-SHOT PROMPT (EXPERT) ===")
# print(few_shot_prompt_expert)
# print("=== END FEW-SHOT PROMPT ===\n")

text_expert = nlg.generate(context_expert)
text_layman = nlg.generate(context_layman)

print("Technical explanation (for a clinician or researcher):")
print(text_expert)
print("\nShort explanation for a patient:")
print(text_layman)


Top-k ranked features: [('worst area', 0.06724261426271144), ('worst concave points', 0.05740842121890832), ('mean concave points', 0.04227830540602773), ('worst radius', 0.03956915260512325), ('worst perimeter', 0.03026387093604719)]

================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Your task is to generate a faithful natural language explanation based ONLY on the provided factors.

You are generating an explanation for a CLINICIAN or DATA SCIENTIST.
General rules:
- The FINAL EXPLANATION MUST BE WRITTEN ONLY IN ENGLISH.
- Do NOT invent exact numeric percentages, thresholds or counts (like '4%', 'three times higher') that are not explicitly present in the input. Use only qualitative terms such as 'larger', 'smaller', 'high', 'low' when describing magnitude, unless the number is given.
- Do NOT introduce new features or variables that are not listed in the i

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Your task is to generate a faithful natural language explanation based ONLY on the provided factors.

You are generating an explanation for a NON-EXPERT patient.
General rules:
- The FINAL EXPLANATION MUST BE WRITTEN ONLY IN ENGLISH.
- Do NOT invent exact numeric percentages, thresholds or counts (like '4%', 'three times higher') that are not explicitly present in the input. Use only qualitative terms such as 'larger', 'smaller', 'high', 'low' when describing magnitude, unless the number is given.
- Do NOT introduce new features or variables that are not listed in the input.
- Stay faithful to the provided factors and their directions (supports / contradicts / neutral).
Additional guidelines for layman explanations:
- Use simple, calm English that a person without medical or technical background can understan

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


Technical explanation (for a clinician or researcher):
The model predicts a benign tumor because several morphological features have small positive contributions to the malignancy score. The worst area, concave points, mean concave points, radius, and perimeter all exhibit supportive attributes with relatively low magnitude. These combined small positive contributions outweigh any opposing effects and collectively push the overall prediction towards benignity.

Short explanation for a patient:
The model suggests a low risk of breast cancer because the lump has a more benign shape and size, which are typical characteristics of harmless findings. The texture measurements also indicate that this lump is less likely to be cancerous, as its structure is consistent with what we see in non-cancerous cases. Overall, these factors collectively point towards a lower risk of breast cancer.


## Chain-of-Thought NLG (technical + simple)

In [5]:
from src.nlg.cot_generator import ChainOfThoughtGenerator
from src.nlg.ollama_client import ollama_llm_call

# CoT generator also uses the same NLGConfig and Ollama client
cot_gen = ChainOfThoughtGenerator(config.nlg, llm_call_fn=ollama_llm_call)

# CoT explanation for the expert audience
context_cot_expert = {
    **base_context,
    "prediction": pred_expert_en,
}

text_cot_expert = cot_gen.generate(context_cot_expert)

print("=== Chain-of-Thought (technical) ===")
print(text_cot_expert)

# CoT explanation for a simplified prediction label (same mechanism)
context_cot_layman = {
    **base_context,
    "prediction": pred_layman_en,
}

# --- Print one example CoT prompt ---
# cot_prompt = cot_gen.build_cot_prompt(context_cot_expert)
# print("=== CHAIN-OF-THOUGHT PROMPT (EXPERT) ===")
# print(cot_prompt)
# print("=== END CHAIN-OF-THOUGHT PROMPT ===\n")

text_cot_layman = cot_gen.generate(context_cot_layman)

print("\n=== Chain-of-Thought (simple) ===")
print(text_cot_layman)



================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Your task is to reason step by step and then provide a clear explanation of the prediction.

Rules:
- The final explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respect the directions 'supports' / 'contradicts' / 'neutral'.
- Use the step-by-step reasoning only as an internal tool; the final explanation should be a clean paragraph.

--- Input context ---
Prediction: benign tumor
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


--- Reasoning (you can think step by step here) ---
Think step by ste

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"


=== Chain-of-Thought (technical) ===
The predicted outcome is that the tumor is benign. The top factors contributing to this prediction are all related to the characteristics of the worst areas and features of the tumor, including its concave points, radius, and perimeter. These factors all support the prediction of a benign tumor, indicating that the tumor's shape and size are consistent with a non-malignant growth. Overall, the combination of these factors suggests that the tumor is likely to be benign.

================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Your task is to reason step by step and then provide a clear explanation of the prediction.

Rules:
- The final explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respe

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



=== Chain-of-Thought (simple) ===
Based on the analysis, it appears that the patient has a low risk of developing breast cancer. The most significant factors contributing to this prediction are the worst area, worst concave points, mean concave points, worst radius, and worst perimeter, all of which have positive SHAP values indicating support for the prediction. These features suggest that the patient's tumor is less aggressive and has a more benign morphology, leading to a lower risk of cancer development. Overall, the combination of these factors collectively justifies the predicted low risk of breast cancer.


## Self-Consistency NLG (aggregated explanation)

In [6]:
from src.nlg.self_consistency_generator import SelfConsistencyGenerator

# Self-consistency generator: multiple chains + aggregated explanation
sc_gen = SelfConsistencyGenerator(
    config.nlg,
    n_chains=3,               # you can increase this, but latency will go up
    llm_call_fn=ollama_llm_call,
)

# Technical / expert-level aggregated explanation
context_sc_expert = {
    **base_context,
    "prediction": pred_expert_en,
}

# --- Print the prompt used for the first reasoning chain ---
# sc_prompt_chain1 = sc_gen.build_chain_prompt(context_sc_expert, chain_id=1)
# print("=== SELF-CONSISTENCY PROMPT (CHAIN 1, EXPERT) ===")
# print(sc_prompt_chain1)
# print("=== END SELF-CONSISTENCY PROMPT ===\n")

text_sc_expert = sc_gen.generate(context_sc_expert)

print("=== Self-Consistency (aggregated, technical) ===")
print(text_sc_expert)

# Simple / layman-style aggregated explanation (using a simpler prediction label)
context_sc_layman = {
    **base_context,
    "prediction": pred_layman_en,
}

text_sc_layman = sc_gen.generate(context_sc_layman)

print("\n=== Self-Consistency (aggregated, simple) ===")
print(text_sc_layman)



================ LLM INPUT PROMPT ================
You are an explainable AI assistant (reasoning path 1).
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Explain this model prediction independently.

Rules:
- The explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respect the 'supports' / 'contradicts' / 'neutral' directions.
- Provide a concise explanation in 3–6 sentences.

--- Input context ---
Prediction: benign tumor
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


Now write the explanation:

================ END OF PROMPT ==================



INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



================ LLM INPUT PROMPT ================
You are an explainable AI assistant (reasoning path 2).
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Explain this model prediction independently.

Rules:
- The explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respect the 'supports' / 'contradicts' / 'neutral' directions.
- Provide a concise explanation in 3–6 sentences.

--- Input context ---
Prediction: benign tumor
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


Now write the explanation:

================ END OF PROMPT ==================



INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



================ LLM INPUT PROMPT ================
You are an explainable AI assistant (reasoning path 3).
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Explain this model prediction independently.

Rules:
- The explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respect the 'supports' / 'contradicts' / 'neutral' directions.
- Provide a concise explanation in 3–6 sentences.

--- Input context ---
Prediction: benign tumor
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


Now write the explanation:

================ END OF PROMPT ==================



...


================ LLM INPUT PROMPT ================
You are an explainable AI assistant (reasoning path 3).
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Explain this model prediction independently.

Rules:
- The explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respect the 'supports' / 'contradicts' / 'neutral' directions.
- Provide a concise explanation in 3–6 sentences.

--- Input context ---
Prediction: low risk of breast cancer
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


Now write the explanation:

================ END OF PROMPT ==================



INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You are given several independent explanations generated for the same model prediction.
Your task is to aggregate them into a single, coherent explanation.

Rules:
- Write the final explanation ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the context or in the explanations.
- Focus on points that appear in at least two explanations, or are clearly consistent.
- Provide a clear explanation in 3–6 sentences.

--- Prediction & top factors (context) ---
Prediction: low risk of breast cancer
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


--- Independent explanations ---

Explanation 1:
The model predicts a low risk of breas

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



=== Self-Consistency (aggregated, simple) ===
The model predicts a low risk of breast cancer due to several supportive factors. The presence of a worst area and worst concave points suggests that the tumor's shape and size are not concerning, aligning with a lower risk of breast cancer. Additionally, the mean concave points, radius, and perimeter all contribute positively to this outcome, indicating that these aspects are benign or even beneficial for breast cancer risk assessment. Overall, the combination of these supportive factors leads the model to conclude that the individual has a low risk of developing breast cancer.


## Full pipeline NLG (Few-Shot vs CoT vs Self-Consistency)

In [7]:
# End-to-end test: let the pipeline handle XAI + NLG
# We only switch the NLG technique: few_shot / cot / self_consistency

for tech in ["few_shot", "cot", "self_consistency"]:
    print("\n==============================")
    print(f"TECHNIQUE = {tech}")
    print("==============================")

    res = pipeline.explain_instance(
        instance,
        method="shap",
        generate_text=True,
        technique=tech,      # This selects the NLG generator inside the pipeline
        audience="expert",   # Audience is only used by FewShotGenerator
    )

    print("Reported technique:", res.get("nlg_technique"))
    print("Reported generator:", res.get("nlg_generator"))
    print("\nGenerated text:\n")
    print(res.get("generated_text", "")[:800])  # print only the first 800 characters
    print("\nValidation:", res.get("validation"))


INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30
INFO:src.utils:[x] Layer 3: Generating text
INFO:src.utils:  generator: few_shot
INFO:src.utils:  technique: few_shot
INFO:src.utils:  audience: expert



TECHNIQUE = few_shot

================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Your task is to generate a faithful natural language explanation based ONLY on the provided factors.

You are generating an explanation for a CLINICIAN or DATA SCIENTIST.
General rules:
- The FINAL EXPLANATION MUST BE WRITTEN ONLY IN ENGLISH.
- Do NOT invent exact numeric percentages, thresholds or counts (like '4%', 'three times higher') that are not explicitly present in the input. Use only qualitative terms such as 'larger', 'smaller', 'high', 'low' when describing magnitude, unless the number is given.
- Do NOT introduce new features or variables that are not listed in the input.
- Stay faithful to the provided factors and their directions (supports / contradicts / neutral).
Additional guidelines for expert explanations:
- It is acceptable to refer to 'features', 'contributions', and

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:src.utils:[x] Layer 4: Validating
INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30
INFO:src.utils:[x] Layer 3: Generating text
INFO:src.utils:  generator: cot
INFO:src.utils:  technique: cot
INFO:src.utils:  audience: expert


Reported technique: few_shot
Reported generator: few_shot

Generated text:

The model predicts a positive outcome because several morphological features have small to moderate positive contributions to the overall prediction. The worst area, concave points, mean concave points, radius, and perimeter all provide supporting evidence for this class, with each feature making a relatively similar-sized contribution. These combined positive contributions drive the final prediction towards a positive outcome, with no dominant opposing factors present in the top features.

Validation: {'sum_conservation_valid': False, 'computed_sum': 0.97, 'clarity_score': 74.66666666666666}

TECHNIQUE = cot

================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Your task is to reason step by step and then provide a clear explanation of the prediction.

Rules:
- The final explanation mus

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:src.utils:[x] Layer 4: Validating
INFO:src.utils:[x] Layer 1: Generating explanations
INFO:src.utils:  method: shap
INFO:src.utils:[x] Layer 2: Normalizing
INFO:src.utils:  features: 30
INFO:src.utils:[x] Layer 3: Generating text
INFO:src.utils:  generator: self_consistency
INFO:src.utils:  technique: self_consistency
INFO:src.utils:  audience: expert


...


================ LLM INPUT PROMPT ================
You are an explainable AI assistant (reasoning path 3).
You receive model predictions and feature attributions (for example, SHAP or LIME values).
Explain this model prediction independently.

Rules:
- The explanation must be written ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the input.
- Respect the 'supports' / 'contradicts' / 'neutral' directions.
- Provide a concise explanation in 3–6 sentences.

--- Input context ---
Prediction: 1
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


Now write the explanation:

================ END OF PROMPT ==================



INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"



================ LLM INPUT PROMPT ================
You are an explainable AI assistant.
You are given several independent explanations generated for the same model prediction.
Your task is to aggregate them into a single, coherent explanation.

Rules:
- Write the final explanation ONLY in English.
- Do NOT invent exact numeric percentages or counts that are not in the input.
- Do NOT introduce new features that are not listed in the context or in the explanations.
- Focus on points that appear in at least two explanations, or are clearly consistent.
- Provide a clear explanation in 3–6 sentences.

--- Prediction & top factors (context) ---
Prediction: 1
Explanation method: shap
Top factors:
- worst area = 0.067 (supports)
- worst concave points = 0.057 (supports)
- mean concave points = 0.042 (supports)
- worst radius = 0.040 (supports)
- worst perimeter = 0.030 (supports)


--- Independent explanations ---

Explanation 1:
The model predicts that this shape is a circle, and it's likel

INFO:httpx:HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
INFO:src.utils:[x] Layer 4: Validating


Reported technique: self_consistency
Reported generator: self_consistency

Generated text:

The model predicts that this shape is a circle due to several features that suggest high symmetry and lack of significant irregularities. The presence of worst area, worst concave points, mean concave points, worst radius, and worst perimeter all contribute to the overall circularity of the shape, indicating consistent curvature and no sharp edges or corners.

Validation: {'sum_conservation_valid': False, 'computed_sum': 0.97, 'clarity_score': 64.0}
